# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
**Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.**
**W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.**
**Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.**
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
#1.Wczytaj obraz rice.png.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/'

fileName = "rice.png"
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
# W dwóch pętlach for, dla okien o ustalonym wymiarze  W  (potęga 2), oblicz średnią:
# pętle powinny mieć krok  W ,
# wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
# przydatny operator to // - dzielenie całkowitoliczbowe (floor division).

Funkcja AdaptBinarization wykonuje adaptacyjną binaryzację obrazu, dzieląc obraz na mniejsze bloki o rozmiarze
𝑤
×
𝑤
. Dla każdego bloku obliczana jest średnia wartość jasności, która następnie jest stosowana jako lokalny próg binaryzacji dla pikseli w danym bloku.

In [ ]:
def AdaptBinarization(img, w: int) -> None:
    # Obliczanie tablicy wartości progów (średnich wartości dla bloków o rozmiarze w x w)
    threshold_tab = np.zeros((img.shape[0] // w, img.shape[1] // w))
    for i in range(0, img.shape[0], w):  # Przechodzenie przez każdy blok wzdłuż wysokości
        for j in range(0, img.shape[1], w):  # Przechodzenie przez każdy blok wzdłuż szerokości
            # Obliczanie średniej jasności dla bieżącego bloku w x w który znajduje się na pozycji i j
            threshold_tab[i // w, j // w] = np.average(img[i : i + w, j : j + w])

    # Binaryzacja obrazu na podstawie obliczonych progów w tablicy threshold_arr
    for i in range(img.shape[0]):  # Iterowanie po każdym wierszu obrazu
        for j in range(img.shape[1]):  # Iterowanie po każdej kolumnie obrazu
            # Zastosowanie progu dla bieżącego bloku w x w
            img[i, j] = 255 if img[i, j] > threshold_tab[i // w, j // w] else 0

    # Wyświetlenie obrazu po binaryzacji
    plt.imshow(img, "gray")
    plt.axis("off")
    plt.show()


In [ ]:
rice = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
AdaptBinarization(rice, 32)

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego piksela wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

**SPRAWDZAMY CZY PIKSEL JEST W ROGU **

In [ ]:
def isHorizontalBoundary(i, _):
    # Sprawdzenie, czy i jest równe 0 lub 255
    if i in [0, 255]:
        return True  # Zwraca True, gdy i wynosi 0 lub 255
    return False  # Zwraca False dla innych wartości i


In [ ]:
def isVerticalBoundary(_, j):
    # Sprawdzenie, czy j jest równe 0 lub 255
    if j in [0, 255]:
        return True # Zwraca True, gdy i wynosi 0 lub 255
    return False # Zwraca False dla innych wartości i


Wielkość okna, oznaczona jako W, odnosi się do rozmiaru obszaru (zwanego oknem lub blokiem), który jest rozważany wokół danego piksela podczas operacji przetwarzania obrazu. Okno to kwadratowy fragment obrazu o wymiarach
𝑊
×
𝑊
W×W pikseli, na którym wykonywane są określone operacje (takie jak uśrednianie jasności, interpolacja czy binaryzacja).

Przykład zastosowania
Jeśli W wynosi 15, to dla każdego analizowanego piksela rozważany jest kwadratowy blok 15 × 15 pikseli wokół niego. W kontekście binaryzacji i filtracji wielkość okna określa zasięg, na którym średnia wartość jasności lub inne statystyki są obliczane w sąsiedztwie danego piksela, co wpływa na lokalne dopasowanie parametrów przetwarzania obrazu.

In [ ]:
def IsWall(i, j, W, image): #i - wiersz j - kolumna w - wielkość okna image - obraz
    #Sprawdzamy blisko której granicy leży piksel
    result = []
    if i < W / 2:
        result.append("u")  # piksel blisko górnej granicy obrazu
    if j < W / 2:
        result.append("l")  # piksel blisko lewej granicy obrazu
    if i >= image.shape[0] - W / 2:
        result.append("d")  # piksel blisko dolnej granicy obrazu
    if j >= image.shape[1] - W / 2:
        result.append("r")  # piksel blisko prawej granicy obrazu
    return result

In [ ]:
IsWall(2, 2, 5, cv2.imread('rice.png', cv2.IMREAD_GRAYSCALE))   #przykładowe wywołanie

In [ ]:
def Interpolation(image, W: int):
    # Obliczanie tablicy średnich wartości jasności dla bloków o rozmiarze W x W
    avg_brightness_arr = np.zeros((image.shape[0] // W, image.shape[1] // W))
    for i in range(0, image.shape[0], W):
        for j in range(0, image.shape[1], W):
            avg_brightness_arr[i // W, j // W] = np.average(image[i : i + W, j : j + W]) #obliczamy średnią jasność

    # Parametr tolerancji jasności
    l = 10
    # Tworzenie nowej tablicy obrazu o tych samych wymiarach
    dummy = np.zeros(image.shape)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            iT = i // W  # Indeks poziomy bieżącego bloku
            jT = j // W  # Indeks pionowy bieżącego bloku

            # Sprawdzenie, czy piksel znajduje się przy granicy obrazu
            walls = IsWall(i, j, W, image)

            #1
            # 1. Róg obrazu
            if len(walls) == 2:
                brightness = avg_brightness_arr[iT, jT]
                test = brightness

             #2
             # Ściana obrazu (jedna granica)
            elif len(walls) == 1:
                if walls[0] in ["u", "d"]:  # Górna lub dolna granica
                    firstj = int(j - W / 2) // W
                    secondj = firstj + 1

                    dYup = j % W
                    dYdown = W - dYup

                    # Pobieranie średnich jasności dla bieżącego i sąsiedniego bloku
                    t11 = avg_brightness_arr[iT, firstj]
                    t12 = avg_brightness_arr[iT, secondj]

                    # Interpolacja w pionie
                    interpolation_y1 = (t11 * dYup / W) + (t12 * dYdown / W)
                    test = int(interpolation_y1)

                else:  # Lewa lub prawa granica
                    firsti = int(i - W / 2) // W
                    secondi = firsti + 1

                    dXleft = i % W
                    dXright = W - dXleft

                    # Pobieranie średnich jasności dla bieżącego i sąsiedniego bloku
                    t11 = avg_brightness_arr[firsti, jT]
                    t12 = avg_brightness_arr[secondi, jT]

                    # Interpolacja w poziomie
                    interpolation_x1 = (t11 * dXleft / W) + (t12 * dXright / W)
                    test = int(interpolation_x1)


            # Piksel nie znajduje się przy żadnej granicy
            else:
                firsti = int(i - W / 2) // W
                secondi = firsti + 1
                firstj = int(j - W / 2) // W
                secondj = firstj + 1

                dXleft = i % W
                dXright = W - dXleft
                dYup = j % W
                dYdown = W - dYup

                # Interpolacja dwukierunkowa
                t11 = avg_brightness_arr[firsti, firstj]
                t12 = avg_brightness_arr[firsti, secondj]
                t21 = avg_brightness_arr[secondi, firstj]
                t22 = avg_brightness_arr[secondi, secondj]

                interpolation_x1 = (t11 * dXleft / W) + (t12 * dXright / W)
                interpolation_x2 = (t21 * dXleft / W) + (t22 * dXright / W)

                # Ostateczna interpolacja
                interpolated_value = (interpolation_x1 * dYup / W) + (interpolation_x2 * dYdown / W)
                test = interpolated_value
            dummy[i, j] = 255 if image[i, j] > test + l else 0


    # Wyświetlenie zbinaryzowanego obrazu
    plt.imshow(dummy, "gray")
    plt.axis("off")
    plt.show()

    return dummy



In [ ]:
rice = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
Interpolation(rice, 16)

In [ ]:
def showImgGetHist(path: str, show_img: bool = False, show_hist: bool = False):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img_hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    return img, img_hist

def localBinarization(image: np.ndarray, w: int = 15) -> None:
    img = image.copy()
    x, y = img.shape
    for i in range(w // 2, y - w // 2):
        for j in range(w // 2, x - w // 2):
            window = image[i - w // 2 : i + w // 2 + 1, j - w // 2 : j + w // 2 + 1]
            threshold = np.mean(window)
            img[i, j] = 255 if image[i, j] > threshold else 0
    return img



def SauvolBinarization(image: np.ndarray, R: float = 128, k: float = 0.15, w:int = 15):

    img = image.copy()
    x, y = img.shape
    for i in range(w // 2, y - w // 2):
        for j in range(w // 2, x - w // 2):
            window = image[i - w // 2 : i + w // 2, j - w // 2 : j + w // 2]
            threshold = np.mean(window) * (1 - k * ((np.std(window) / R) - 1))
            img[i, j] = 255 if img[i, j] > threshold else 0
    return img

In [ ]:
# plot all
def plotAll(img):
    fig, axs = plt.subplots(5, 1, figsize=(40, 40))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original')
    axs[0].axis('off')
    # plot otsu
    rice_otsu = cv2.threshold(rice, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    axs[1].imshow(rice_otsu, cmap='gray')
    axs[1].set_title('Otsu')
    axs[1].axis('off')
    # plot Local Binarization
    rice_local = localBinarization(img)
    axs[2].imshow(rice_local, cmap='gray')
    axs[2].set_title('Local Binarization')
    axs[2].axis('off')

    # plot sauvola
    rice_sauvola = SauvolBinarization(img)
    axs[3].imshow(rice_sauvola, cmap='gray')
    axs[3].set_title('Sauvola Binarization')
    axs[3].axis('off')
    # plot interpolation
    rice_interpolation = Interpolation(img, 16)
    axs[4].imshow(rice_interpolation, cmap='gray')
    axs[4].set_title('Interpolation')
    axs[4].axis('off')
    plt.show()
    AdaptBinarization(img, 16)


plotAll(cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE))